In [6]:
!pip list | grep -e 'numpy' -e 'scipy' -e 'pandas' -e 'scikit-learn'

numpy                              1.15.4   
numpydoc                           0.8.0    
pandas                             0.24.1   
scikit-learn                       0.20.3   
scipy                              1.2.0    


## 前処理

In [7]:
import numpy as np
import pandas as pd
# サンプルのデータセットを作成
df = pd.DataFrame(
    {
        'A': [1, np.nan, 3, 4, 5],
        'B': [6, 7, 8, np.nan, 10],
        'C': [11, 12, 13, 14, 15]
    }
)
df

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


### 欠損値の扱い

In [8]:
# 欠損値の除去はpandasのメソッドで処理
# isnullメソッドでTrueの行が落とされる
df.isnull()

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


In [9]:
df_dropped = df.dropna()
df_dropped

,A,B,C
0,1.0,6.0,11
2,3.0,8.0,13
4,5.0,10.0,15


In [10]:
# 欠損値の補完
from sklearn.impute import SimpleImputer

# 平均値で欠損値を補完するインスタンスを作成する
# 平均は 'mean'、中央値は 'median'、最頻値は 'most_frequent', 固定値は 'constant' で指定する
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

# 欠損値を補完
imp.fit(df)
imp.transform(df)  # Numpy配列が返却される

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

### カテゴリ変数のエンコーディング

In [11]:
df_category = pd.DataFrame(
    {
        'A': [1, 2, 3, 4, 5],
        'B': ['a', 'b', 'a', 'b', 'c']
    }
)
df_category

,A,B
0,1,a
1,2,b
2,3,a
3,4,b
4,5,c


In [13]:
# カテゴリ変数を数値変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_category['B'])
le.transform(df_category['B'])

array([0, 1, 0, 1, 2])

In [14]:
# 元の値と変換先の対応
le.classes_

array(['a', 'b', 'c'], dtype=object)

In [15]:
# カテゴリ変数をOne-hotエンコーディング
from sklearn.preprocessing import OneHotEncoder
df_ohe = df_category.copy()
le = LabelEncoder()
# カテゴリ変数を数値変換
df_ohe['B'] = le.fit_transform(df_ohe['B'])
# Ohe-hotエンコーダのインスタンス化
ohe = OneHotEncoder(categorical_features=[1])
# One-hotエンコーディング
ohe.fit_transform(df_ohe).toarray()

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [1., 0., 0., 3.],
       [0., 1., 0., 4.],
       [0., 0., 1., 5.]])

In [23]:
# pandasのメソッドを使う
df_ohe_2 = df_category.copy()
df_ohencoded = pd.get_dummies(df_category.loc[:, 'B'], prefix='B')
df_ohe_merged = pd.concat([df_category['A'], df_ohencoded], axis=1)
df_ohe_merged

,A,B_a,B_b,B_c
0,1,1,0,0
1,2,0,1,0
2,3,1,0,0
3,4,0,1,0
4,5,0,0,1


### 特徴量の正規化

In [28]:
import pandas as pd
df = pd.DataFrame(
    {
        'A': [1, 2, 3, 4, 5],
        'B': [100, 200, 400, 500, 800]
    }
)
df

,A,B
0,1,100
1,2,200
2,3,400
3,4,500
4,5,800


In [29]:
# 分散正規化 ... 平均0, 標準偏差1になるように各特徴量を調節
from sklearn.preprocessing import StandardScaler
# 分散正規化のインスタンスを生成
stdsc = StandardScaler()
# 分散正規化を実行
stdsc.fit(df)
stdsc.transform(df)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


array([[-1.41421356, -1.22474487],
       [-0.70710678, -0.81649658],
       [ 0.        ,  0.        ],
       [ 0.70710678,  0.40824829],
       [ 1.41421356,  1.63299316]])

In [30]:
# 最小最大正規化 ... 最小値0, 最大値1になるように各特徴量を調節
from sklearn.preprocessing import MinMaxScaler
# 分散正規化のインスタンスを生成
mmsc = MinMaxScaler()
# 分散正規化を実行
mmsc.fit_transform(df)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


array([[0.        , 0.        ],
       [0.25      , 0.14285714],
       [0.5       , 0.42857143],
       [0.75      , 0.57142857],
       [1.        , 1.        ]])